In [1]:
import h5py
import numpy as np
import os 
import pandas as pd 
import json
import glob 
from pathlib import Path
import matplotlib.pyplot as plt
import math
from scipy.optimize import least_squares

from datetime import datetime
import shutil


In [2]:

### this cell generates all of the filepaths 

## change here 
base_folder =  "/ceph/behrens/mingyu_zhu/vHPC_mPFC"

behaviour_folder = f"{base_folder}/data/raw_data/behaviour"
roi_folder = f"{base_folder}/data/preprocessed_data/SLEAP_ROIs"
sleap_folder = f"{base_folder}/data/preprocessed_data/SLEAP"
preprocessing_params_folder = f"{base_folder}/code/preprocessing/params"
maze_params_folder = f"{preprocessing_params_folder}/maze_params"
metadata_folder = f"{base_folder}/data/raw_data/metadata"
maze_params_folder = f"{base_folder}/code/preprocessing_maze_registration/maze_params"


In [8]:
### find the files to read from, you should have identified the odd files and RENAME them by adding "_original" to the end of the filename 

for filename in os.listdir(behaviour_folder):
    if '_original.csv' in filename:
        print(f"{filename}")


mz10_pinstate_2024-10-23-172819_original.csv
mz07_pinstate_2024-10-22-142824_original.csv
mz09_pinstate_2024-11-11-093452_original.csv
mz05_pinstate_2024-10-23-172819_original.csv
mz07_pinstate_2024-11-11-093452_original.csv
mz07_pinstate_2024-10-23-164824_original.csv
mz06_pinstate_2024-10-22-142824_original.csv
mz06_pinstate_2024-10-23-164824_original.csv


In [15]:
## option for spliting tracking data (.h5 file and roi file) into two 
#### this is needed if forgot to reclick camera recording when stopping pycontrol and ephys 

### manually put in the things that would need to be filled in: 
mouseID = 'mz09'
err_pinstate_id = '2024-11-11-093452_original' ### the original pinstate file 
target_pinstate_id = [ '2024-11-11-093452',  '2024-11-11-093620'] ### the two new pinstate handler -- the second one should start before the pycontrol/ephys recording for the next session.

### things that probably don't need change
target_node = 'head_back'
fmt = '%H%M%S'
samp_freq_vid = 60 

## find the files to read from 
err_sleap_fp = glob.glob(f"{sleap_folder}/*{mouseID}_{err_pinstate_id}*")[0]
err_roi_fp = glob.glob(f"{roi_folder}/*{mouseID}_{err_pinstate_id}*")[0]
err_pinstate_fp = f"{behaviour_folder}/{mouseID}_pinstate_{err_pinstate_id}.csv"

### the new filepaths for the split pinstate, sleap, and roi files in list [fp1, fp2]
output_pinstate_fp = [f"{behaviour_folder}/{mouseID}_pinstate_{target_pinstate_id[0]}.csv",f"{behaviour_folder}/{mouseID}_pinstate_{target_pinstate_id[1]}.csv"]
output_sleap_fp = [f"{sleap_folder}/{mouseID}_{target_pinstate_id[0]}.predicted_.h5",f"{sleap_folder}/{mouseID}_{target_pinstate_id[1]}.predicted_.h5"]
output_roi_fp = [f"{roi_folder}/{mouseID}_{target_pinstate_id[0]}.csv",f"{roi_folder}/{mouseID}_{target_pinstate_id[1]}.csv"]


err_pinstate = pd.read_csv(err_pinstate_fp,header=None)
err_roi =  pd.read_csv(err_roi_fp)

shutil.copyfile(err_sleap_fp, output_sleap_fp[0])
shutil.copyfile(err_sleap_fp, output_sleap_fp[1])

length_diff = samp_freq_vid*int((datetime.strptime(target_pinstate_id[1].split('-')[-1],fmt) - datetime.strptime(target_pinstate_id[0].split('-')[-1],fmt)).total_seconds())

with h5py.File(output_sleap_fp[0], 'r+') as f:
    data = f['tracks'][:,:,:,:length_diff] 
    del f['tracks'] 
    f.create_dataset('tracks', data=data) 
with h5py.File(output_sleap_fp[1], 'r+') as f:
    data = f['tracks'][:,:,:,length_diff:] 
    del f['tracks'] 
    f.create_dataset('tracks', data=data) 

output_pinstate_file_1 = err_pinstate[:length_diff]
output_pinstate_file_2 = err_pinstate[length_diff:]
output_pinstate_file_1.to_csv(output_pinstate_fp[0], index = False, header = None)
output_pinstate_file_2.to_csv(output_pinstate_fp[1], index = False, header = None)

output_roi_file_1 = err_roi[:length_diff]
output_roi_file_2 = err_roi[length_diff:]
output_roi_file_1.to_csv(output_roi_fp[0])
output_roi_file_2.to_csv(output_roi_fp[1])


### checking output 
print(f"splitting session {err_pinstate_id} into {target_pinstate_id[0]} and {target_pinstate_id[1]}")
print(f"pinstate length before splitting: {len(err_pinstate)}; after splitting: {len(output_pinstate_file_1) + len(output_pinstate_file_2)}")
print(f"roi length before splitting: {len(err_roi)}; after splitting: {len(output_roi_file_2) + len(output_roi_file_1)}")


splitting session 2024-11-11-093452_original into 2024-11-11-093452 and 2024-11-11-093620
pinstate length before splitting: 78496; after splitting: 78496
roi length before splitting: 78496; after splitting: 78496
